<a href="https://colab.research.google.com/github/krishna7356/pandas/blob/gen-AI/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("Hello world")

Hello world


In [2]:
from transformers import AutoTokenizer

# Load GPT-2 tokenizer
gpt2_tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Load BERT-base-uncased tokenizer
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Example usage
text = "Hello, how are you?"

print("GPT-2 Tokenized:", gpt2_tokenizer.tokenize(text))
print("BERT Tokenized:", bert_tokenizer.tokenize(text))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

GPT-2 Tokenized: ['Hello', ',', 'Ġhow', 'Ġare', 'Ġyou', '?']
BERT Tokenized: ['hello', ',', 'how', 'are', 'you', '?']
